# UTSA CS 3793/5233: Project

**Castillo Cesar - kzl544**

## Topic: Body Pose Estimation - Yoga Pose Detection

## Description

This is an application of pose detection and estimation using the Blaze Pose model and a classifer to classify yoga poses using the support vector classifier from the sklearn library. The Blaze Pose model can be implemented through a library, well known as media pipe.

## Import Libraries

In [ ]:
!pip install mediapipe
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
from sklearn.svm import SVC
from google.colab.patches import cv2_imshow

In [ ]:
# path = "/DATASET/TRAIN/plank" # complete the whole path if you need to. E.g. C:/Users/... or "/content/drive/MyDrive/Colab Notebooks/..."

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Preparing Dataset for Pose Estimation

In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks
data = []
for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data = pd.DataFrame(columns = data) # Empty dataset

count = 0

for img in os.listdir(path):

        temp = []

        img = cv2.imread(path + "/" + img)

        imageWidth, imageHeight = img.shape[:2]

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        blackie = np.zeros(img.shape) # Blank image

        results = pose.process(imgRGB)

        if results.pose_landmarks:

                # mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on image

                mpDraw.draw_landmarks(blackie, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blackie

                landmarks = results.pose_landmarks.landmark

                for i,j in zip(points,landmarks):

                        temp = temp + [j.x, j.y, j.z, j.visibility]

                data.loc[count] = temp

                count +=1

        cv2_imshow(img)

        cv2_imshow(blackie)

        #cv2.waitKey(100)

data.to_csv("dataset3.csv") # save the data as a csv file

## Creating the Pose Estimation model

In [ ]:
data = pd.read_csv("dataset3.csv")
print(data.head())
X = data.iloc[:,:132]
Y = data['target']
model = SVC(kernel = 'poly')
model.fit(X,Y)
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils
# path = "enter path here"
imagePath = path + "/" + img
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = pose.process(imgRGB)
if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        for j in landmarks:
                temp = temp + [j.x, j.y, j.z, j.visibility]
        y = model.predict([temp])
        if y == 0:
            asan = "plank"
        else:
            asan = "goddess"
        print(asan)
        cv2.putText(img, asan, (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.imshow("image",img)

## Reference
Gupta Ayush . **Human Pose Estimation Using Machine Learning in Python** [Internet]. 2021 October 26. Available from: https://www.analyticsvidhya.com/blog/2021/10/human-pose-estimation-using-machine-learning-in-python/

